In [5]:
import torch
import torch.nn as nn
import numpy as np
from copy import deepcopy

device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
class RBF(nn.Module):
    def __init__(self):
        super(RBF, self).__init__()
        torch.cuda.manual_seed(0)
        
        self.rbf_clt = self.init_clt()
        self.rbf_std = self.init_std()
        

    def init_clt(self):
        return nn.Parameter(torch.rand(1))

    def init_std(self):
        return nn.Parameter(torch.rand(1))
    
    def rbf(self, x, cluster, std):
        return torch.exp(-(x - cluster) * (x - cluster) / 2 * (std * std))
    
    
    def forward(self, x):     
        
        x = self.rbf(x, self.rbf_clt, self.rbf_std)
    
        return x

In [7]:
class RBFnetwork(nn.Module):
    def __init__(self, timelag):
        super(RBFnetwork, self).__init__()
        torch.cuda.manual_seed(0)
        device = "cuda" if torch.cuda.is_available() else "cpu"
        
        self.timelag = timelag
        self.init_weight = nn.Parameter(torch.rand(self.timelag))
        self.rbf_list = [RBF().to(device) for i in range(self.timelag)]
        
    def forward(self, x):

        for j in range(self.timelag):
            if j ==0:
                y = sum([self.init_weight[i] * self.rbf_list[i](x[j]) for i in range(self.timelag)])    
            else:
                y = torch.cat([y, sum([self.init_weight[i] * self.rbf_list[i](x[j]) for i in range(self.timelag)])])
        
        return y

In [8]:
def restore_parameters(model, best_model):
    '''Move parameter values from best_model to model.'''
    for params, best_params in zip(model.parameters(), best_model.parameters()):
        params.data = best_params
        
def train_RBFlayer(model, input_, target, lr, epochs, lookback = 5, device = device):
    model.to(device)
    loss_fn = nn.MSELoss(reduction='mean')
    optimizer = torch.optim.Adam(model.parameters(), lr = lr)
    
    train_loss_list = []
    
    best_it = None
    best_model = None
    best_loss = np.inf
    
    target_list = []
    
    for j in range(len(target) - 2):
        target_list.append((target[j+2] - target[j])/2)
    
    loss_list = []
    cause_list = []
    for epoch in range(epochs):
        cause = model(input_)
        cause_list.append(cause)
        grad = []
        
        
        for i in range(len(cause) - 2):
            grad.append((cause[i+2] - cause[i])/2)
        
        loss1 = sum([loss_fn(grad[i], target_list[i]) for i in range(len(grad))])
        loss2 = sum([loss_fn(cause[i], target[i]) for i in range(len(input_))])
        
        loss = loss1 + loss2
        
        loss.backward()
        optimizer.step()
        model.zero_grad()
        
        loss_list.append(loss)
        mean_loss = loss / len(grad)
        train_loss_list.append(mean_loss)
        
        if mean_loss < best_loss:
            best_loss = mean_loss
            best_it = epoch
            best_model = deepcopy(model)
            
        elif (epoch - best_it) == lookback:
            if verbose:
                print('Stopping early')
            break
        print("epoch {} cause loss {} :".format(epoch, loss / len(input_)))
        print('gradient loss :', loss1/len(grad))
        print('value loss :', loss2/len(input_))
                
    best_cause = cause_list[best_it]    
    restore_parameters(model, best_model)

    return best_model, loss_list, best_cause

In [9]:
def data_split(X, cause, target, timelag, device = device):
    input_cause = []
    input_target = []
    Y = []

    for i in range(len(X) - (timelag + 1)):
        input_cause.append(X[cause].values[i: i + timelag])
        input_target.append(X[target].values[i: i + timelag])
        Y.append([X[target][i + timelag + 1]])

    return torch.tensor(input_cause, device=device).float(), torch.tensor(input_target,device=device).float(), torch.tensor(Y, device=device).float()


In [10]:
import pandas as pd
df = pd.read_csv('C:/Users/chanyoung/Desktop/Neural-GC-master/lorenz_96_10_10_1000.csv')
X2d = df[['a','b']]
torch.manual_seed(1234)
input_cause, input_target, Y = data_split(X2d, 'a', 'b', 100)

In [11]:
input_cause1 = input_cause[0]

In [12]:
input_cause1

tensor([ 9.9017e-02,  2.3833e+00,  6.1237e+00,  8.7908e+00,  8.6702e+00,
         8.3957e+00,  6.3754e+00,  7.8874e-01, -3.0315e+00, -3.0010e+00,
        -1.9771e+00, -1.6429e+00, -1.6446e+00, -1.7082e+00, -1.4593e+00,
         5.0648e-01,  2.0741e+00,  1.0970e+00, -2.6947e-01, -4.8096e-01,
         2.9342e-01,  1.3812e+00,  2.6800e+00,  4.1593e+00,  4.5540e+00,
         1.2930e+00,  3.8564e-01,  2.4397e+00,  2.4273e+00, -3.3669e-01,
        -3.8871e+00, -3.5459e+00, -1.4380e+00, -1.1335e+00,  2.1363e+00,
         6.8083e+00,  9.3094e+00,  9.3140e+00,  9.4318e+00,  8.4774e+00,
         3.1353e+00, -3.7522e+00, -6.4836e+00, -5.6106e+00, -4.7806e+00,
        -3.5820e+00, -1.6181e+00,  9.8395e-01,  2.8974e+00,  3.3778e+00,
         2.8026e+00,  3.2360e+00,  3.9812e+00,  5.3865e+00,  6.0212e+00,
         5.1376e+00,  1.2646e+00, -2.7911e+00, -4.7048e+00, -4.2577e+00,
        -2.4119e+00, -1.4249e+00, -5.2749e-02,  2.8018e+00,  6.2285e+00,
         9.0628e+00,  1.0530e+01,  1.1011e+01,  1.0

In [14]:
import time

cause_list = []

print("0 번째 time series")
start = time.time()
model = RBFnetwork(100)
best_model, loss_list, best_cause = train_RBFlayer(model, input_cause1, input_cause1,0.01, 100, device)
cause_list.append(best_cause.cpu().detach().numpy())
print("time :", time.time() - start)
print('-------------------------------------------------------------------------------------------')


0 번째 time series
epoch 0 cause loss 699.0578002929688 :
gradient loss : tensor(84.4160, device='cuda:0', grad_fn=<DivBackward0>)
value loss : tensor(616.3301, device='cuda:0', grad_fn=<DivBackward0>)
epoch 1 cause loss 670.5623779296875 :
gradient loss : tensor(81.3850, device='cuda:0', grad_fn=<DivBackward0>)
value loss : tensor(590.8051, device='cuda:0', grad_fn=<DivBackward0>)
epoch 2 cause loss 642.7049560546875 :
gradient loss : tensor(78.4138, device='cuda:0', grad_fn=<DivBackward0>)
value loss : tensor(565.8594, device='cuda:0', grad_fn=<DivBackward0>)
epoch 3 cause loss 615.4962158203125 :
gradient loss : tensor(75.5035, device='cuda:0', grad_fn=<DivBackward0>)
value loss : tensor(541.5028, device='cuda:0', grad_fn=<DivBackward0>)
epoch 4 cause loss 588.9454345703125 :
gradient loss : tensor(72.6551, device='cuda:0', grad_fn=<DivBackward0>)
value loss : tensor(517.7435, device='cuda:0', grad_fn=<DivBackward0>)
epoch 5 cause loss 563.0623168945312 :
gradient loss : tensor(69.869

epoch 45 cause loss 54.25782012939453 :
gradient loss : tensor(10.1154, device='cuda:0', grad_fn=<DivBackward0>)
value loss : tensor(44.3448, device='cuda:0', grad_fn=<DivBackward0>)
epoch 46 cause loss 51.45127868652344 :
gradient loss : tensor(9.6578, device='cuda:0', grad_fn=<DivBackward0>)
value loss : tensor(41.9866, device='cuda:0', grad_fn=<DivBackward0>)
epoch 47 cause loss 48.89447784423828 :
gradient loss : tensor(9.2306, device='cuda:0', grad_fn=<DivBackward0>)
value loss : tensor(39.8485, device='cuda:0', grad_fn=<DivBackward0>)
epoch 48 cause loss 46.570960998535156 :
gradient loss : tensor(8.8320, device='cuda:0', grad_fn=<DivBackward0>)
value loss : tensor(37.9156, device='cuda:0', grad_fn=<DivBackward0>)
epoch 49 cause loss 44.4647216796875 :
gradient loss : tensor(8.4606, device='cuda:0', grad_fn=<DivBackward0>)
value loss : tensor(36.1734, device='cuda:0', grad_fn=<DivBackward0>)
epoch 50 cause loss 42.5602912902832 :
gradient loss : tensor(8.1149, device='cuda:0', gr

epoch 90 cause loss 26.788768768310547 :
gradient loss : tensor(4.1566, device='cuda:0', grad_fn=<DivBackward0>)
value loss : tensor(22.7153, device='cuda:0', grad_fn=<DivBackward0>)
epoch 91 cause loss 26.712350845336914 :
gradient loss : tensor(4.1468, device='cuda:0', grad_fn=<DivBackward0>)
value loss : tensor(22.6485, device='cuda:0', grad_fn=<DivBackward0>)
epoch 92 cause loss 26.63559341430664 :
gradient loss : tensor(4.1379, device='cuda:0', grad_fn=<DivBackward0>)
value loss : tensor(22.5805, device='cuda:0', grad_fn=<DivBackward0>)
epoch 93 cause loss 26.55861473083496 :
gradient loss : tensor(4.1297, device='cuda:0', grad_fn=<DivBackward0>)
value loss : tensor(22.5115, device='cuda:0', grad_fn=<DivBackward0>)
epoch 94 cause loss 26.481502532958984 :
gradient loss : tensor(4.1222, device='cuda:0', grad_fn=<DivBackward0>)
value loss : tensor(22.4417, device='cuda:0', grad_fn=<DivBackward0>)
epoch 95 cause loss 26.40439796447754 :
gradient loss : tensor(4.1154, device='cuda:0',

In [15]:
plt.plot(cause_list[0])

NameError: name 'plt' is not defined

In [ ]:
plt.plot(cause_list[0])
plt.plot(input_cause[0].cpu().detach().numpy())

In [24]:
print(best_model.rbf_list[0].rbf_clt)
print(best_model.rbf_list[0].rbf_std)


Parameter containing:
tensor([0.6339], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.0456], device='cuda:0', requires_grad=True)


In [25]:
print(best_model.rbf_list[1].rbf_clt)
print(best_model.rbf_list[1].rbf_std)

Parameter containing:
tensor([0.5955], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.6505], device='cuda:0', requires_grad=True)
